# Market Basket Analysis

Association Analysis

Get pandas and MLxtend code imported and read the data:

In [1]:
#imports
import pandas as pd #Python data analysis library
import numpy as np #Python scientific computing
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

Citation - “The Instacart Online Grocery Shopping Dataset 2017”, Accessed from https://www.instacart.com/datasets/grocery-shopping-2017"

In [2]:
#import dataset
trainDf = pd.read_csv("../data/raw/order_products__train.csv")
orderDf = pd.read_csv("../data/raw/orders.csv")
productDf = pd.read_csv("../data/raw/products.csv")

We will merge the prior and train dataset to get the complete order dataset. We will use append() for doing so.

In [3]:
priorDf = pd.read_csv("../data/raw/order_products__prior.csv")
trainDf = trainDf.append(priorDf,ignore_index = True)

For counting each product, we can assign reordered column as 1 

In [4]:
trainDf['reordered'] = 1 

In [5]:
productCountDf = trainDf.groupby("product_id",as_index = False)["order_id"].count()

In [6]:
#Top 100 most frequently purchased products
topLev = 100

#Here order_id is the count so we need to sort the data frame w.r.t order_id
productCountDf = productCountDf.sort_values("order_id",ascending = False)

topProdFrame = productCountDf.iloc[0:topLev,:]
topProdFrame = topProdFrame.merge(productDf,on = "product_id")
productId= topProdFrame.loc[:,["product_id"]]

Now we will filter the orders and get orders containting the the most frequently purchased products

In [7]:
df = trainDf[0:0]
for i in range(0,99):
    pId = productId.iloc[i]['product_id'] 
    stDf = trainDf[trainDf.product_id == pId ]
    df = df.append(stDf,ignore_index = False)

In [8]:
df.head()

,order_id,product_id,add_to_cart_order,reordered
115,226,24852,2,1
156,473,24852,2,1
196,878,24852,2,1
272,1042,24852,1,1
297,1139,24852,1,1


 Now we need to consolidate the items into 1 transaction per row with each product 1 hot encoded. Each row will represent an order and each column will represent product_id. If the cell value is '1' say (i,j) then ith order contains jth product.

In [9]:
basket = df.groupby(['order_id', 'product_id'])['reordered'].sum().unstack().reset_index().fillna(0).set_index('order_id')

In [10]:
# Convert the units to 1 hot encoded values
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1    

In [11]:
basket_sets = basket.applymap(encode_units)

In [12]:
basket_sets.head()

product_id,196,3957,4210,4605,4799,4920,5077,5450,5785,5876,...,46667,46906,46979,47144,47209,47626,47766,48679,49235,49683
order_id,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
basket_sets.size

241667217

Now that the data is structured properly, we can generate frequent item sets that have a support of at least 1% 

In [14]:
# Build up the frequent items
frequent_itemsets = apriori(basket_sets, min_support=0.01, use_colnames=True)

In [15]:
frequent_itemsets

,support,itemsets
0,0.015279,[196]
1,0.016088,[3957]
2,0.015144,[4210]
3,0.031514,[4605]
4,0.015439,[4799]
5,0.035537,[4920]
6,0.025855,[5077]
7,0.021353,[5450]
8,0.020226,[5785]
9,0.037381,[5876]


The final step is to generate the rules with their corresponding support, confidence and lift:

In [16]:
# Create the rules
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)
rules

,antecedants,consequents,support,confidence,lift
0,(26209),(47626),0.060080,0.197723,3.001750
1,(47626),(26209),0.065869,0.180345,3.001750
2,(47766),(21903),0.075468,0.175281,1.699910
3,(21903),(47766),0.103112,0.128289,1.699910
4,(47209),(27966),0.090483,0.121389,2.077938
5,(27966),(47209),0.058418,0.188018,2.077938
6,(24852),(21903),0.201259,0.108683,1.054029
7,(21903),(24852),0.103112,0.212133,1.054029
8,(13176),(21903),0.161785,0.133208,1.291881
9,(21903),(13176),0.103112,0.209007,1.291881


 We can also see several where the confidence is high as well.or instance, we can see that there are quite a few rules with a high lift value which means that it occurs more frequently than would be expected given the number of transaction and product combinations. 

We can filter the dataframe using standard pandas code. In this case, look for a large lift (2) and high confidence (.1):

In [17]:
rules[ (rules['lift'] >= 2) &
       (rules['confidence'] >= 0.1) ]

,antecedants,consequents,support,confidence,lift
0,(26209),(47626),0.060080,0.197723,3.001750
1,(47626),(26209),0.065869,0.180345,3.001750
4,(47209),(27966),0.090483,0.121389,2.077938
5,(27966),(47209),0.058418,0.188018,2.077938
21,(39275),(21137),0.043024,0.238274,2.110653
22,(47626),(47766),0.065869,0.160244,2.123337
23,(47766),(47626),0.075468,0.139862,2.123337
42,(21137),(27966),0.112891,0.128940,2.207206
43,(27966),(21137),0.058418,0.249174,2.207206
